
## Import những thư viện cần thiết


In [ ]:
import pandas as pd # thư viện làm việc với dữ liệu dạng bảng
import numpy as np # thư viện tối ưu hóa tính toán trên mảng
import matplotlib.pyplot as plt # thư viện vẽ biểu đồ
import seaborn as sns # thư viện vẽ biểu đồ 
import math # thư viện toán có sẵn
from sklearn.model_selection import train_test_split # hàm chia dữ liệu
from sklearn.linear_model import LinearRegression # mô hình hồi quy tuyến tính
import re
from collections import Counter
from sklearn.ensemble import RandomForestRegressor #Mô hình RandomForest cho hồi quy.


#**Data Understanding**

In [ ]:
# Load dữ liệu lên để sử dụng
df = pd.read_csv('euro_it_salary_2020.csv', index_col=0)
df

In [ ]:
# Xem ngẫu nhiên n dòng dữ liệu
df.sample(10)

In [ ]:
# Kích thước dữ liệu
df.shape

In [ ]:
# Kiểu dữ liệu
df.info()

In [ ]:
# Mô tả thống kê
df.describe()

In [ ]:
# Thống kê cho biến định tính
df[['Gender', 'TotYrsOfExp', 'MainTech']].describe()

In [ ]:
df['Gender'].unique()

In [ ]:
# Số lượng dữ liệu duy nhất
df.nunique()


#**Data Preprocessing**

In [ ]:
# Biểu đồ xem nan trong dataframe
plt.figure(figsize=(10, 8))
sns.heatmap(df.isna())
plt.show()

Kiểm tra những dòng missing nhiều cột xóa nó đi. 

In [ ]:
# Các điểm dữ liệu missing tất cả các cột
nan_many = df[df['Age'].isna() & df['TotYrsOfExp'].isna() & df['MainTech'].isna() & df['AnualSalary'].isna()]
print(nan_many.shape)
nan_many

In [ ]:
# Xóa bỏ các điểm dữ liệu trên
df.drop(index=nan_many.index, inplace=True)
df = df.reset_index(drop=True)
df[df['Age'].isna() & df['TotYrsOfExp'].isna() & df['MainTech'].isna() & df['AnualSalary'].isna()]


#### Xử lý column TotYrsOfExp

**Xử lý invalid data**

In [ ]:
# Kiểm tra dữ liệu thuộc tính TotYrsOfExp
df.TotYrsOfExp.unique()

In [ ]:
# Thay dấu "," thành dấu "."
df.TotYrsOfExp = df['TotYrsOfExp'].str.strip()
df.TotYrsOfExp = df['TotYrsOfExp'].str.replace(',', '.')
df.TotYrsOfExp.unique()

In [ ]:
# Những vị trí sai số năm kinh nghiệm
iter = df.loc[df.TotYrsOfExp.str.len() > 4]
iter

In [ ]:
# Gán trực tiếp dành cho dữ liệu sai ít
# df.loc[df['TotYrsOfExp'].str.len() > 4, 'TotYrsOfExp'] = np.array([11, 15, 6, 0])

In [ ]:
# Hàm xử lý invalid data.
def fixing(datum):
  if (not isinstance(datum, str) and math.isnan(datum)): # Nếu nó là nan
    return np.nan
  elif len(str(datum)) > 4: 
    results = [e for e in re.split(r'[^0-9]', datum) if e != ''] # Chuỗi xác định những cái ko phải số.
    if len(results) == 0:
      return 0
    return max(map(int, results))
  return datum

**Hàm fixing:** Nhận các giá trị qua phương thức apply, xử lý rồi trả về giá trị đã xử lý ngay tại điểm data đó.

In [ ]:
# Apply trên tất cả các điểm dữ liệu trên cột
df['TotYrsOfExp'].apply(fixing)

In [ ]:
# Gán lại để cột ban đầu
df['TotYrsOfExp'] = df['TotYrsOfExp'].apply(fixing)

In [ ]:
# Kiểm tra
df['TotYrsOfExp'].unique()

**Xử lý missing data TotYrsOfExp**

In [ ]:
# Xem những điểm missing
df[df['TotYrsOfExp'].isna()]

In [ ]:
# Kiểm tra số lượng missing
df[df['TotYrsOfExp'].isna()].shape

In [ ]:
# Tính mean của column TotYrsExp
mean = np.nanmean(df.TotYrsOfExp.astype('float'))
print(mean)

# Thay thế mean cho nan
df.TotYrsOfExp.fillna(mean, inplace=True)

In [ ]:
# Kiểm tra lại
df.TotYrsOfExp.unique()

**Format về đúng kiểu dữ liệu**
- Xử lý dữ liệu bị sai(invalid data) trước, xử lý nan trước --> Formatting

In [ ]:
# Chuyển data về dạng số. 
df['TotYrsOfExp'] = pd.to_numeric(df['TotYrsOfExp'])
df.info()

**Kiểm tra và xử lý outliers**

In [ ]:
# Biểu đồ xem outliers data
df['Age'].plot(kind='box')

In [ ]:
# Hàm trả về râu dưới và râu trên
def bound(x):
  q3, q1 = np.quantile(x, 0.75), np.quantile(x, 0.25)
  IQR = q3 - q1
  return [q1 - 1.5*IQR, q3 + 1.5*IQR]

In [ ]:
df['TotYrsOfExp'].describe()

In [ ]:
# Biểu đồ hộp trên TotYrsOfExp
plt.figure(figsize=(5, 10))
sns.boxplot(data=df['TotYrsOfExp'])
plt.show()

In [ ]:
# Kiểm tra dữ liệu outliers
upper = bound(df['TotYrsOfExp'])[1]
df[df['TotYrsOfExp'] > upper] 

# Khám phá 

In [ ]:
# Số dòng, cột outliers
df[df['TotYrsOfExp'] > upper].shape

In [ ]:
# Tính số lần xuất hiện của một giá  trị
df['TotYrsOfExp'].value_counts()

In [ ]:
# Gán lại vị trí lỗi
df.loc[df['TotYrsOfExp'] == df['TotYrsOfExp'].max(), 'TotYrsOfExp'] = 10
df['TotYrsOfExp'].max()

In [ ]:
# Check lại biểu đồ
plt.figure(figsize=(5, 10))
sns.boxplot(data=df['TotYrsOfExp'])
plt.show()

In [ ]:
# Xem qua một tí về biểu đồ tương quan
corr = df.corr()
sns.heatmap(data=corr, annot=True)
plt.show()

####Xử lý trên column Age

Cột tuổi thì không có dữ liệu bị lỗi.

In [ ]:
# Kiểm dữ liệu
df['Age'].dtype

In [ ]:
# Các giá trị có trong cột Age
df['Age'].unique()

In [ ]:
# Biểu đồ xem nan
sns.heatmap(df[['Age']].isna())

In [ ]:
# Liệt kê các điểm dữ liệu missing
nan_age = df[df['Age'].isna()]
print(nan_age.shape)
nan_age

In [ ]:
# Biểu đô tương quan 2 biến
plt.figure(figsize=(10, 8))
plt.scatter(df['Age'], df['TotYrsOfExp'])
plt.show()

In [ ]:
# Lấy các giá trị cần dự đoán
bePre = df[df['Age'].isna()][['TotYrsOfExp']].values
bePre, bePre.shape

In [ ]:
# Dữ liệu cho mô hình học (không chưa nan)
X = df[~df['Age'].isna()][['TotYrsOfExp']].values
y = df[~df['Age'].isna()]['Age'].values
X.shape, y.shape

**Mô hình linear**: Age = a*SoNamKinhNghiem + b

In [ ]:
# Thực thi mô hình 
model = LinearRegression().fit(X, y)
print(model.score(X, y))

# Dự đoán cho vị trí nan
predict = model.predict(bePre)
predict

In [ ]:
# Thay giá trị được dự đoán vào data nan của Age
nan_index = df[df['Age'].isna()].index
df.loc[nan_index, 'Age'] = predict

# Kiểm tra nan trong Age
df[df['Age'].isna()]

#### Kiểm tra và xử lý outliers của Age:

In [ ]:
# Biểu đồ xem outliers
plt.figure(figsize=(8, 8))
sns.boxplot(data=df['Age'])
plt.show()

Phần này chọn cách giữ nguyên. Các bạn có thể thử thêm các cách khác và xem thử độ chính xác mô hình có tăng lên không.

In [ ]:
# Xem các giá trị ngoại lệ.
upper = bound(df['Age'])[1]
df[df['Age'] > upper]

In [ ]:
df[df['Age'] > upper].shape

####Xử lý column MainTech

In [ ]:
df['MainTech']

In [ ]:
# dict những tên công nghệ cần thay thế về 1 tên chung. (jS --> javasript, k8s --> kubernetes, ...)
dict_apply_duplicate = {
    r'js': 'javascript',
    r'k8s': 'kubernetes',
    r'kubrrnetes': 'kubernetes',
    r'gcp': 'google cloud platform',
    r'\.net': '.net',
    r'[^\w](\.net)': '.net core',
    'core core': 'core',
    r'frontend:': 'frontend,',
    r'react': 'react.js',
    r'reactjs': 'react.js',
    r'nodejs': 'node.js',
    r'node': 'node.js',
    r'ts': 'typescript',
    r'[\(\)]': ''
}


def doing(data: str):
    deprecated = ['none', '-', '--', 'nothing'] # list những cái giá trị bỏ
    # dưới đây là if data là nan hoặc data nằm trong list những cái bỏ thì mình trả về no_tech
    if (not isinstance(data, str) and math.isnan(data)) or (data in deprecated):
        return ['no_tech']
    
    data = data.lower() # lowercase dữ liệu chuỗi
    # gọi cái dict ở trên
    # thay thế key bằng value có dấu , hai đầu để tiện split
    global dict_apply_duplicate
    for regex, sub in dict_apply_duplicate.items(): # Thay thế các chuỗi viết tắt thành tên đầy đủ theo value của dict
        sub_data = ',%s,' % (sub)
        data = re.compile(regex).sub(sub_data, data)
    
    data = data.replace('/', ',') # thay dấu / thành ,
    data = data.split(',') # split công nghệ theo ,
    data = [datum.strip() for datum in data] # loại bỏ space hai đầu của từng công nghệ
    data = [datum for datum in data if datum != ''] # nếu loại bỏ xong là rỗng thì không lấy 
    data = ['_'.join(datum.split()) for datum in data] # ghép lại theo _
    
    if data == ['aws_hadoop_postgre_typescript']: # đối với trường hợp cụ thể như thế này thì thay thế riêng
        data = data[0].split('_')
        
    return data

**Hàm doing:** Input là các giá trị của cột MainTech thông qua phương thức apply. Xử lý rồi trả về giá trị ngay tại điểm dữ liệu đó.

1. Kiểm tra nếu trong list deprecated hoặc là nan gán "no_tech".
2. Lower case cả chuỗi về chữ thường.
3. Thay thế các chuỗi viết tắt thành tên đầy đủ theo value của dict_apply_duplicate.
4. Thay dấu "/" thành ",".
5. Tách chuỗi theo dấu ","
6. Loại bỏ khoảng trằng " " có ở 2 đầu (vd : " python  ").
7. Loại bỏ các giá trị "".
8. Ghép tên các công nghệ trên 2 từ bởi "_".
9. Trường hợp chuỗi "aws_hadoop_postgre_typescript" phải tách ra thành các công nghệ riêng biệt.
10. Trả về data đã xử lý.


In [ ]:
# Apply trên các điểm dữ liệu
df['MainTech'] = df['MainTech'].apply(doing)

In [ ]:
# Show kết quả
df['MainTech']

####Xử lý column AnualSalary

In [ ]:
# Kiểm tra dữ liệu lỗi
print(type(df['AnualSalary']))
df['AnualSalary'].unique()

In [ ]:
df['AnualSalary'].dtype

In [ ]:
# Xem outliers
sns.boxplot(data=df['AnualSalary'])

In [ ]:
# Xem các outliers
l = df[df['AnualSalary'].notna()]['AnualSalary']
upper = bound(l)[1]
df[df['AnualSalary'] > upper]

In [ ]:
# Khám phá một tí về mức lương
df[df['AnualSalary'] > 163500]

In [ ]:
# Thể các giá trị ngoài lệ phía trên bằng trung bình
df = df[df['AnualSalary'] != df['AnualSalary'].max()]
df['AnualSalary'].max()

In [ ]:
# Xem dữ liệu nan
df[df['AnualSalary'].isna()]

In [ ]:
# Điền nan bằng trung bình
mean = df['AnualSalary'].mean()
df['AnualSalary'].fillna(mean, inplace=True)

In [ ]:
df[df['AnualSalary'].isna()]

In [ ]:
sns.boxplot(data=df['AnualSalary'])

In [ ]:
df = df[df['AnualSalary'] != df['AnualSalary'].max()]
sns.boxplot(df['AnualSalary'])

Ngoài các cách xử lý trên còn có : forward fill, back fill dùng cho timeseries data. --> [Document](https://www.oreilly.com/library/view/learning-pandas/9781787123137/82b10237-8e03-414f-a9aa-b52316f78976.xhtml)

##Binning
Tạo thêm data phục vụ cho việc phân tích dữ liệu.

**Bạn có thể tham khảo thêm: -->** [Doc](https://towardsdatascience.com/data-preprocessing-with-python-pandas-part-5-binning-c5bd5fd1b950)

Binning và explore cho column Age

In [ ]:
# #Tạo nhãn và khoảng bins
labels_age = ['Nguoi tre', 'Trung nien', 'Gia']
bins_age = [0, 30, 45, 70]

# #Tạo column bins
df['Age_bins'] = pd.cut(df['Age'], bins=bins_age, labels=labels_age)
df['Age_bins']

In [ ]:
# # Khám phá thông tin
stat = df[['Age_bins']].value_counts()[:]
explode = (0, 0.1, 0.24)

plt.figure(figsize=(10, 8))
plt.pie(stat, explode=explode, labels=labels_age, autopct='%1.1f%%', shadow=True)
plt.legend()
plt.show()

Binning và explore cho column AnualSalary

In [ ]:
#Tạo khoảng và nhãn bins
labels_salary = ['0 - 20.000', '20.000 - 40.000', '40.000 - 80.000', '80.000 ++']
range_salary = [0, 20000, 40000, 80000, df['AnualSalary'].max()]

#Tạo columns bins
df['Salary_bins'] = pd.cut(df['AnualSalary'], bins=range_salary, labels=labels_salary)
df['Salary_bins']

In [ ]:
# #Khám phá thông tin
stat = df[['Salary_bins']].value_counts()[:]
explode = (0.05, 0.05, 0, 0.05)

plt.figure(figsize=(10, 8))
plt.pie(stat, explode=explode, labels=labels_salary, autopct='%1.1f%%', shadow=True)
plt.legend()
plt.show()

In [ ]:
#Tạo khoảng và nhãn bins
labels_exp = ['Gà Mờ', 'Gà Gà', 'Gà Trống', 'Gà Đầu Đàn', 'Gà Già Đầu']
range_exp = [0, 3, 7, 15, 30, df['TotYrsOfExp'].max()]

#Tạo columns bins
df['exp_bins'] = pd.cut(df['TotYrsOfExp'], bins=range_exp, labels=labels_exp)
df['exp_bins']

In [ ]:
#Khám phá thông tin
stat = df[['exp_bins']].value_counts()[:]
explode = (0.05, 0.02, 0, 0.02, 0.02)

plt.figure(figsize=(10, 8))
plt.pie(stat, explode=explode, labels=labels_exp, autopct='%1.1f%%', shadow=True)
plt.legend()
plt.show()

In [ ]:
# Drop những cột tạo từ bước Binning
df = df.drop(columns=['exp_bins', 'Salary_bins', 'Age_bins'])
df

##Data Normalization
Chuẩn hóa dữ liệu làm cho các thuộc tính có cùng một đơn vị. Việc này giúp cho các thuộc tính có cùng một trọng số khi đưa vào mô hình. Để làm feature input cho một thuật toán thì nên áp dụng cùng một phương pháp normalization cho tất cả các feature. 

In [ ]:
# Normalization column định lượng: (Simple feature scaling)
df['Age'] = df['Age']/df['Age'].max()
df['TotYrsOfExp'] = df['TotYrsOfExp']/df['TotYrsOfExp'].max()
df['AnualSalary'] = df['AnualSalary']/df['AnualSalary'].max()
df[['Age', 'TotYrsOfExp', 'AnualSalary']]

**Bạn có thể tham khảo thêm các cách khác ở đây: -->** [Document](https://www.geeksforgeeks.org/data-normalization-with-pandas/)

##Label encoder
Bước này chuyển dữ liệu định tính --> dữ liệu dạng số để model có thể học được.

####Label encode column Gender
**Chuyển Male --> 1, Female --> 0, Diverser --> 2. Đây là phương pháp label encode.** 


In [ ]:
def encode(datum):
  if (datum == 'Male') | (not isinstance(datum, str) and math.isnan(datum)) | (datum == 'nan'):
    return 1
  if (datum == 'Diverse'):
    return 2
  return 0

In [ ]:
# Apply trên các giá trị cột
df['Gender'] = df['Gender'].apply(encode)

# Kiểm tra lại
df['Gender'].unique()

####One-hot encode column MainTech
Tạo các cột với tên là công nghệ được sử dụng. Nếu một ngừoi có sử dụng thì mang giá trị 1 và ngược lại là 0. Đây là phương pháp One-hot encode bạn có thể search gg để hiểu thêm.

In [ ]:
df['MainTech']

In [ ]:
threshold = 7
list_tech = list()
for i in df['MainTech']:
    for j in i:
        if j: # tránh những trường hợp ''
            list_tech.append(j)

counter = pd.DataFrame(Counter(list_tech).items(), columns=['name', 'count']).sort_values(by='count', ascending=False)
counter

In [ ]:
counter[counter['count'] >= threshold] 

In [ ]:
one_hot_column = counter[counter['count'] >= threshold]['name'].values
foo = [] 
one_hot_list = []
for idx, row in enumerate(df['MainTech']):
    flag = True
    one_row = {'MainTech.{}'.format(tech): 1 if tech in row else 0 for tech in one_hot_column}
    for key, val in one_row.items():
      flag &= val == 0
    if flag:
      foo.append(idx)
    one_hot_list.append(one_row)
    
main_tech_one_hot_df = pd.DataFrame(one_hot_list)
main_tech_one_hot_df

In [ ]:
main_tech_one_hot_df.shape

In [ ]:
# Datafrane cuối cùng, đầu vào của mô hình 
input = pd.concat([df.drop('MainTech', axis=1), main_tech_one_hot_df], axis=1)
input = input.drop(foo).reset_index(drop=True)

In [ ]:
input

**2 phương pháp encode phía trên sẽ có ưu nhược điểm khác nhau khi đưa vào model. -->** [Document tham khảo](https://www.analyticsvidhya.com/blog/2020/03/one-hot-encoding-vs-label-encoding-using-scikit-learn/)

#**Modeling**
+ Mô hình hồi quy ***y = ax + b*** . --> [Document](https://machinelearningcoban.com/2016/12/28/linearregression/)

+ Bước này mình sử dụng thuật toán randomforest dành cho hồi quy. --> [Document](https://gdcoder.com/random-forest-regressor-explained-in-depth/)

In [ ]:
# Drop các dòng nan do label encode xong
input = input.dropna().reset_index(drop=True)
input

In [ ]:
# Lấy tập X, y đầu vào mô hình
y = input['AnualSalary']
input = input.drop(columns='AnualSalary')
names = input.columns.to_list()
X = input[names]

# Kiểm tra các cột đầu vào
print(names)

In [ ]:
# Dạng input mô hình
X.shape, y.shape

In [ ]:
# Chia dữ liệu thành train test
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=3)

Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape

In [ ]:
# Sử dụng mô hình hồi quy tuyến tính của thuật toán randomforest
model = RandomForestRegressor(random_state=3).fit(Xtrain, ytrain)
predict = model.predict(Xtest)

print('Điểm trên tập train : ', model.score(Xtrain, ytrain))
print('Điểm trên tập test :', model.score(Xtest, ytest))